# 最佳实践-百炼官方文档智能助手


## 1. 简介
本文介绍基于百炼平台，构建官方文档助手，结合知识库构建，Assistant API调用，实现全代码创建知识库场景的智能体应用。

### 1.1. 依赖功能
* Assistant API的基础API参数生成能力。
* Assistant API的Function calling能力。

### 1.2. 前期准备
* 已开通百炼服务：[开通阿里云百炼大模型服务产品](https://help.aliyun.com/document_detail/2586399.html?spm=a2c4g.2784257.0.i6)。
* 已创建API_KEY: [获取API-KEY](https://help.aliyun.com/document_detail/2712195.html)。
* 已安装最新版SDK：[安装SDK](https://help.aliyun.com/document_detail/2712193.html)。

## 2. 实现代码

### 2.1. 设置环境变量
安装环境依赖，设置您的API-KEY，替换YOUR_DASHSCOPE_API_KEY为您自己的API key。

In [ ]:
# 安装dashscope SDK。
!pip3 install dashscope

In [ ]:
# 通过环境变量设置API-KEY
%env DASHSCOPE_API_KEY=YOUR_API_KEY

### 2.2. 创建知识库

百炼平台支持管控台页面创建知识索引，以及以代码方式进行创建。
以下代码示例兼容LlamaIndex SDK设计。

In [ ]:
!pip install llama-index-core
!pip install llama-index-llms-dashscope
!pip install llama-index-readers-file
!pip install llama-index-indices-managed-dashscope-custom

In [ ]:
import os
from llama_index.readers.dashscope.base import DashScopeParse
from llama_index.readers.dashscope.utils import ResultType
from llama_index.indices.managed.dashscope import DashScopeCloudIndex, DashScopeCloudRetriever
from llama_index.core import SimpleDirectoryReader
from llama_index.llms.dashscope import DashScope, DashScopeGenerationModels
from typing import cast
from enum import Enum

In [ ]:
def ingest_data(file_folder: str, name: str, category_id="default"):
    # 本地文档上传，并完成文档解析，可以指定百炼数据中心的目录
    parse = DashScopeParse(result_type=ResultType.DASHSCOPE_DOCMIND, category_id=category_id)
    file_extractor = {".pdf": parse, '.doc': parse, '.docx': parse}
    documents = SimpleDirectoryReader(
        file_folder, file_extractor=file_extractor
    ).load_data(num_workers=4)

    # 构建知识索引，完成文档切分、向量化和入库操作
    _ = DashScopeCloudIndex.from_documents(documents, name, verbose=True)
    return documents

In [ ]:
class ReturnType(str, Enum):
    STRING = "str"
    NODE = "node"


def retrieve_nodes(query: str, name: str, return_type: ReturnType = ReturnType.STRING):
    """
    知识库检索函数
    :param query: 用户Query 
    :param name: 百炼知识索引名称
    :param return_type: 支持返回str或者LlamaIndex nodes
    :return: 
    """
    def _format(doc_name=None, title=None, content=None, chunk_index=1):
        return f"[{chunk_index}] 【文档名】{doc_name}\n 【标题】{title}\n 【正文】{content}"

    index = DashScopeCloudIndex(name)
    retriever = index.as_retriever(rerank_min_score=0.3)
    retriever = cast(DashScopeCloudRetriever, retriever)
    print(f"Pipeline id {retriever.pipeline_id}", flush=True)
    nodes = retriever.retrieve(query)
    if return_type == ReturnType.STRING:
        context_str = "\n\n".join([_format(
            doc_name=node.metadata["doc_name"],
            title=node.metadata["title"],
            content=node.get_content(),
            chunk_index=index + 1
        ) for index, node in enumerate(nodes)])
        return context_str
    else:
        return nodes


def query_rag(query: str, name: str):
    """
    结合LLM，构造Query Engine，完成RAG完整链路
    :param query: 用户Query 
    :param name: 百炼知识索引名称
    :return: 
    """
    dashscope_llm = DashScope(
        model_name=DashScopeGenerationModels.QWEN_MAX, api_key=os.environ["DASHSCOPE_API_KEY"]
    )
    index = DashScopeCloudIndex(name)
    query_engine = index.as_query_engine(llm=dashscope_llm)
    response = query_engine.query(query)
    return response

In [ ]:
index_name = "上海AI峰会-百炼文档助手"

"""Step 1: 百炼知识库数据导入"""
docs = ingest_data(
    "/Users/towardsun/Documents/Project/Bailian/Demo", index_name, 
    category_id="cate_ce28ea58687d4db599d974fe5cd7852010001"
)

In [ ]:
"""Step 2: 在线RA检索"""
question = "请问百炼平台如何部署模型"
resp = retrieve_nodes(question, name=index_name)
print(f"Retrieve result:\nQuestion: {question}\nResponse:\n{resp}")

In [ ]:
"""Step 3: 在线RAG完整链路"""
resp = query_rag(question, name=index_name)
print(f"Query result:\nQuestion: {question}\nResponse:\n{resp}")

### 2.3. 创建Assistant API

自定义函数，以下示例包括知识库检索和获取百炼模型训练日志。

In [ ]:
import sys
from dashscope import Assistants, Messages, Runs, Threads
from llama_index.indices.managed.dashscope import DashScopeCloudIndex, DashScopeCloudRetriever
from typing import cast
import json
from http import HTTPStatus

In [ ]:
def retrieve_nodes(rag_query: str):
    """
    环境变量 INDEX_NAME为管控台知识库索引名称。
    :param rag_query: 知识库查询
    :return:
    """
    index = DashScopeCloudIndex("上海AI峰会-百炼文档助手")
    retriever = index.as_retriever()
    retriever = cast(DashScopeCloudRetriever, retriever)
    print(f"Pipeline id {retriever.pipeline_id}", flush=True)
    nodes = retriever.retrieve(rag_query)
    return "\n\n".join([node.get_content() for node in nodes])


def get_train_info(user_id):
    """
    MOCK接口，进行代码测试；
    :param user_id: 用户ID信息
    :return:
    """
    answer = f"{user_id}"
    model_list = ['qwen-max-sft-v1']
    for model in model_list:
        answer += '\nmodel_name: ' + model + '\ntrain_acc ' + str(86.44) + ' \n training time' + '21 hours 40 mintues'

    return str(answer)


function_mapper = {
    "retrieve_nodes": retrieve_nodes,
    "get_train_info": get_train_info
}

In [ ]:
def create_assistant():
    # create assistant with information
    assistant = Assistants.create(
        model="qwen-max",
        name='smart helper',
        description='A tool helper.',
        instructions='You are a helpful assistant. When asked a question, use tools wherever possible.',
        tools=[
            {
                'type': 'quark_search'
            },
            {
                'type': 'function',
                'function': {
                    'name': 'retrieve_nodes',
                    'description': '用于获取与百炼平台相关的信息，包括模型训练、部署，应用搭建、API-KEY等',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'rag_query': {
                                'type': 'str',
                                'description': '百炼相关问题'
                            },
                        },
                        'required': ['rag_query']
                    }
                }
            },
            {
                'type': 'function',
                'function': {
                    'name': 'get_train_info',
                    'description': '用于获取用户部署在百炼平台上用数据额外训练的模型情况。包含SFT模型的具体情况',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'user_id': {
                                'type': 'str',
                                'description': '用户id'
                            },
                        },
                        'required': ['user_id']
                    }
                }
            }
        ],
    )

    if assistant.status_code != HTTPStatus.OK:
        print('Create Assistant failed, status code: %s, code: %s, message: %s' % (
            assistant.status_code, assistant.code, assistant.message))
        sys.exit()
    else:
        print('Create Assistant success, id: %s' % assistant.id)

    return assistant


def create_message(thread_id, content):
    message = Messages.create(thread_id, content=content)
    if message.status_code != HTTPStatus.OK:
        print('Create Message failed, status code: %s, code: %s, message: %s' % (
            message.status_code, message.code, message.message))
        sys.exit()
    else:
        print('Create Message success, id: %s' % message.id)
    return message


def create_thread():
    thread = Threads.create()
    # check result is success.
    if thread.status_code != HTTPStatus.OK:
        print('Create Thread failed, status code: %s, code: %s, message: %s' % (
            thread.status_code, thread.code, thread.message))
        sys.exit()
    else:
        print('Create Thread success, id: %s' % thread.id)
    return thread


def create_run(thread_id, assistant_id):
    run = Runs.create(thread_id, assistant_id=assistant_id)
    if run.status_code != HTTPStatus.OK:
        print('Create Assistant failed, status code: %s, code: %s, message: %s' % (
            run.status_code, run.code, run.message))
    else:
        print('Create Assistant success, id: %s' % run.id)
    return run

In [ ]:
def send_message(assistant, message=''):
    # create a thread.
    thread = create_thread()

    # create a message.
    create_message(thread_id=thread.id, content=message)

    # create run
    response = Runs.create(thread.id, assistant_id=assistant.id, stream=True)
    content_str = ""
    for event, run in response:
        if event == "thread.message.delta":
            content_str += run.delta.content.text.value
            print(content_str, flush=True)
    print(run)

    # wait for run completed or requires_action
    run_status = Runs.wait(run.id, thread_id=thread.id)
    print('插件调用前：')

    if run_status.status == 'failed':
        print('run failed:')
        print(run_status.last_error)

    # if prompt input tool result, submit tool result.
    if run_status.required_action:
        f = run_status.required_action.submit_tool_outputs.tool_calls[0].function
        func_name = f['name']
        param = json.loads(f['arguments'])
        print(f"Function name {func_name}, parameters {param}")
        if func_name in function_mapper:
            output = function_mapper[func_name](**param)
        else:
            output = ""

        tool_outputs = [{
            'tool_call_id': run_status.required_action.submit_tool_outputs.tool_calls[0].id,
            'output': output
        }]

        responses = Runs.submit_tool_outputs(run.id,
                                             thread_id=thread.id,
                                             tool_outputs=tool_outputs,
                                             stream=True)
        content_str = ""
        for event, run in responses:
            if event == "thread.message.delta":
                content_str += run.delta.content.text.value
                print(content_str, flush=True)

        # should wait for run completed
        run_status = Runs.wait(run.id, thread_id=thread.id)

    run_status = Runs.get(run.id, thread_id=thread.id)
    # print(run_status)
    # verify_status_code(run_status)

    # get the thread messages.
    msgs = Messages.list(thread.id)
    # print(msgs)
    # print(json.dumps(msgs, default=lambda o: o.__dict__, sort_keys=True, indent=4))

    print("运行结果:")
    for message in msgs['data'][::-1]:
        print("content: ", message['content'][0]['text']['value'])
    print("\n")

In [ ]:
demo_assistant = create_assistant()

In [ ]:
send_message(assistant=demo_assistant, message="请一句话概括爱因斯坦的成就？")

In [ ]:
send_message(assistant=demo_assistant, message="请问如何获取API KEY？")

In [ ]:
send_message(assistant=demo_assistant, message="请问用户ID 114578的用户，模型训练结果如何？")